# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/asyncio/0.1.69/lib/netstandard2.0/AsyncIO.dll"
#r @"../../../../../../../.nuget/packages/netmq/4.0.1.13/lib/netstandard2.1/NetMQ.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/CommonFSharp.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        // use runtime = new NetMQ.NetMQRuntime ()
        use request = new NetMQ.Sockets.RequestSocket ()
        request.Connect $"tcp://127.0.0.1:{port}"

        let msg = NetMQ.NetMQMessage ()
        msg.Append json

        NetMQ.OutgoingSocketExtensions.SendMultipartMessage (request, msg)
        let result = NetMQ.ReceivingSocketExtensions.ReceiveMultipartMessage (request, 10)
        // let! result = NetMQ.AsyncReceiveExtensions.ReceiveMultipartMessageAsync (request, 10) |> Async.AwaitTask
        let resultString = result |> Seq.map (fun x -> x.ConvertToString ()) |> String.concat ""
        trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.FrameCount: {result.FrameCount} / resultString.Length: {resultString.Length}") getLocals
        return Some resultString
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let! ct =
        cancellationToken
        |> Option.defaultValue System.Threading.CancellationToken.None
        |> Async.mergeCancellationTokenWithDefaultAsync

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if port = 13805 && availablePort <> port then
            let pingObj = {| Ping = true |}
            let! pingResult = pingObj |> sendObj port
            inbox.Post port
        else
            let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"
            // let commandsPath = compilerPath </> "compiler/supervisor/commands"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={availablePort}"
                        CancellationToken = Some ct
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: tcp://*:{availablePort}"
                            then inbox.Post availablePort
                        }
                    }
            trace Debug (fun () -> $"startSupervisor / exitCode: {exitCode} / result: {result}") getLocals
    }, ct)

    let! serverPort = compiler.Receive ()

    let request = new NetMQ.Sockets.SubscriberSocket ()
    request.SubscribeToAnyTopic ()
    request.Connect $"tcp://127.0.0.1:{serverPort + 1}"

    let rec loop i = async {
        let result = NetMQ.ReceivingSocketExtensions.ReceiveMultipartMessage (request, 10)
        // let! result = NetMQ.AsyncReceiveExtensions.ReceiveMultipartMessageAsync (request, 10) |> Async.AwaitTask
        return
            result
            |> Seq.map (fun x -> x.ConvertToString ())
            |> String.concat ""
            |> FSharp.Json.Json.deserialize<ClientErrorsRes>
    }

    return serverPort, loop |> FSharp.Control.AsyncSeq.initInfiniteAsync, (request :> System.IDisposable)
}

## getFileUri

In [ ]:
let inline getFileUri path =
    $"file:///{path |> String.trimStart [| '/' |]}"

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

## buildFile

In [ ]:
let inline buildFile timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let stream, disposable = FileSystem.watchDirectory true fileDir
    use _ = disposable

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    let! serverPort, errors, disposable = awaitCompiler port cancellationToken
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, FileSystemChange.Changed (path, Some content) when path = $"{fileName}.fsx" -> Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> String.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> String.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, true, error)
            | TracedError data ->
                Some (data.message, true, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, false, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, true, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, fatal, error) ->
            None, Some (message, fatal, error)
        )

    let outputSeq =
        [fsxContentSeq; errorsSeq]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], None), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, firstErrorTicks) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, firstErrorTicks
                | None, Some (message, fatal, error) ->
                    fsxContentResult,
                    (message, error) :: errors,
                    firstErrorTicks
                    |> Option.defaultWith (fun () ->
                        if fatal
                        then System.DateTime.MinValue.Ticks
                        else System.DateTime.Now.Ticks
                    )
                    |> Some
                | _ -> fsxContentResult, errors, firstErrorTicks
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, firstErrorTicks) ->
            let firstErrorMs =
                firstErrorTicks
                |> Option.map (fun t -> System.TimeSpan(System.DateTime.Now.Ticks - t).TotalMilliseconds)
            trace Debug (fun () -> $"buildFile / fsxContent: {fsxContent} / errors: {errors} / firstErrorMs: {firstErrorMs}") getLocals
            match fsxContent, errors, firstErrorMs with
            | None, [], _ -> true
            | None, _, Some firstErrorMs when firstErrorMs < 2000. -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.map (fun (fsxContent, errors, _) ->
            fsxContent, errors |> List.distinct
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! fileOpenResult = fileOpenObj |> sendObj serverPort

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (Option.flatten >> Option.defaultValue (None, []))
}

## persistCode

In [ ]:
let inline persistCode timeout cancellationToken code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! code |> FileSystem.writeAllTextAsync mainPath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "spiral"}
packages:
    |core-
    fsharp
modules:
    main
"""
    do! spiprojCode |> FileSystem.writeAllTextAsync spiprojPath

    let disposable = newDisposable (fun () ->
        ()
        // tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore |> Async.RunSynchronously
    )

    return mainPath, disposable
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> buildFile timeout cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "let rec closure0 () () : int32 =
    0
let v0 : (unit -> int32) = closure0()
()
",
        ["main.spi:
Global inl/let statements should all return functions known at parse time."]
    )
)

00:00:00 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0650-5971-7150-7fcb38d42e40 / result: { CreationTime = 2023-08-27 6:50:59 AM
  Exists = true }
00:00:00 #2 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:00 #4 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:00 #5 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:00 #6 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:00 #7 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:00 #8 [Debug] sendJson / port: 13807 / json: {"FileOpen"

In [ ]:
//// test

"""inl app () =
    fsharp.console.write_line "error"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "error"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:02 #15 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-0204-0410-09f1f243d207 / result: { CreationTime = 2023-08-27 6:51:02 AM
  Exists = true }
00:00:02 #16 [Debug] runWithTimeoutAsync / timeout: 60
00:00:02 #17 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:02 #18 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:02 #19 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:02 #20 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:02 #21 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:02 #22 [Debug] sendJson / port: 13807 / json: {"F

In [ ]:
//// test

""
|> buildCode 4000 None
|> Async.runWithTimeout 4000
|> _equal None

00:00:04 #27 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-0382-8288-86b6771c1415 / result: { CreationTime = 2023-08-27 6:51:03 AM
  Exists = true }
00:00:04 #28 [Debug] runWithTimeoutAsync / timeout: 60
00:00:04 #29 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:04 #30 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:04 #31 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:04 #32 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:04 #33 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:04 #34 [Debug] sendJson / port: 13807 / json: {"F

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["An attempt to divide by zero has been detected at compile time."]
    )
)

00:00:08 #41 [Error] runWithTimeoutAsync / timeout: 4000
00:00:08 #42 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-0808-0833-06aede4450e4 / result: { CreationTime = 2023-08-27 6:51:08 AM
  Exists = true }
00:00:08 #43 [Debug] runWithTimeoutAsync / timeout: 60
00:00:08 #44 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:08 #45 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:08 #46 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:08 #47 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:08 #48 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\releas

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "()
",
        ["main.spi:
Expected: uppercase variable"]
    )
)

00:00:10 #54 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-0991-9190-9ab1ea6499c4 / result: { CreationTime = 2023-08-27 6:51:09 AM
  Exists = true }
00:00:10 #55 [Debug] runWithTimeoutAsync / timeout: 60
00:00:10 #56 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:10 #57 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:10 #58 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:10 #59 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:10 #60 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:10 #61 [Debug] sendJson / port: 13807 / json: {"F

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["main.spi:
Expected: whitespace"]
    )
)

00:00:12 #68 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-1175-7550-7bc39e5654fd / result: { CreationTime = 2023-08-27 6:51:11 AM
  Exists = true }
00:00:12 #69 [Debug] runWithTimeoutAsync / timeout: 60
00:00:12 #70 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:12 #71 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:12 #72 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:12 #73 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:12 #74 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:12 #75 [Debug] sendJson / port: 13807 / json: {"F

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["Cannot apply a forall with a term."]
    )
)

00:00:13 #79 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-1284-8416-8358b6fff514 / result: { CreationTime = 2023-08-27 6:51:12 AM
  Exists = true }
00:00:13 #80 [Debug] runWithTimeoutAsync / timeout: 60
00:00:13 #81 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:13 #82 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:13 #83 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:13 #84 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:13 #85 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:13 #86 [Debug] sendJson / port: 13807 / json: {"F

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 8000 None
|> Async.runWithTimeout 8000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        ["The main function should not have a forall."]
    )
)

00:00:14 #91 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-1465-6573-6eb8dc2aacd5 / result: { CreationTime = 2023-08-27 6:51:14 AM
  Exists = true }
00:00:15 #92 [Debug] runWithTimeoutAsync / timeout: 60
00:00:15 #93 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-880> }
00:00:15 #94 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:15 #95 [Debug] buildFile / fsxContent:  / errors: [] / firstErrorMs:
00:00:15 #96 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:15 #97 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:15 #98 [Debug] sendJson / port: 13807 / json: {"F

## getFileTokenRange

In [ ]:
let inline getFileTokenRange timeout cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> FileSystem.readAllTextAsync
    let lines = code |> String.split [| '\n' |]

    let port =
        if fullPath |> String.startsWith (System.IO.Path.GetTempPath ())
            && fullPath |> String.contains "Microsoft.DotNet.Interactive.App" |> not
        then 13807
        else 13805

    let! serverPort, outputs, disposable = awaitCompiler port cancellationToken
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult = fileTokenRangeObj |> sendObj serverPort

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode timeout cancellationToken code
    use _ = disposable
    return! mainPath |> getFileTokenRange timeout cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange 8000 None
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0|])

00:00:21 #103 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-2145-4595-4a030ae81b58 / result: { CreationTime = 2023-08-27 6:51:21 AM
  Exists = true }
00:00:21 #104 [Debug] runWithTimeoutAsync / timeout: 60
00:00:21 #105 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-338> }
00:00:22 #106 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:22 #107 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:22 #108 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:22 #109 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl main () = ()","uri":"file:///C:\\Users\\i574n\\App

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange 8000 None
|> Async.runWithTimeout 8000
|> Option.flatten
|> _equal (Some [|0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0|])

00:00:24 #111 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230827-0651-2463-6382-6dd12dadd571 / result: { CreationTime = 2023-08-27 6:51:24 AM
  Exists = true }
00:00:25 #112 [Debug] runWithTimeoutAsync / timeout: 60
00:00:25 #113 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13807"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-596> }
00:00:25 #114 [Debug] > Server bound to: tcp://*:13807 & tcp://*:13808
00:00:25 #115 [Debug] > pwd: C:\home\git\polyglot\apps\spiral
00:00:25 #116 [Debug] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:25 #117 [Debug] sendJson / port: 13807 / json: {"FileOpen":{"spiText":"inl main () = 1i32","uri":"file:///C:\\Users\\i574n\\A

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.Mandatory>] BuildFile of string * string
    | Timeout of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | BuildFile _ -> nameof Arguments.BuildFile
            | Timeout _ -> nameof Arguments.Timeout

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let inputPath, outputPath =
        match argsMap.[nameof Arguments.BuildFile] with
        | [ Arguments.BuildFile (inputPath, outputPath) ] -> Some (inputPath, outputPath)
        | _ -> None
        |> Option.get

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 30000

    async {
        let! outputCode, errors = inputPath |> buildFile timeout None
        
        errors
        |> List.map snd
        |> List.iter (fun error ->
            trace Error (fun () -> $"main / error: {error}") getLocals
        )

        match outputCode with
        | Some outputCode ->
            do! outputCode |> FileSystem.writeAllTextAsync outputPath
            return 0
        | None ->
            return 1
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0